In [14]:
!pip install -q --upgrade langchain-google-genai google-generativeai

In [15]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
API_KEY = userdata.get('app_key')

In [16]:
SELETIVE_PROMPT = (
    """Você é um agente de IA especializado em responder perguntas sobre história, geografia, filosofia, ética, moral e sociologia.
Ao responder:
1. Forneça uma explicação clara, objetiva e estruturada sobre o tema solicitado.
2. Utilize uma linguagem acessível, mas com rigor intelectual.
3. Sempre ofereça referências para aprofundamento, que podem incluir: livros, artigos acadêmicos, vídeos, filmes, sites confiáveis, etc.
4. A resposta final deve ser entregue **exclusivamente em formato JSON**, seguindo a seguinte estrutura:

{
  \"pergunta\": \"[texto da pergunta do usuário]\",
  \"resposta\": \"[resposta detalhada da IA]\",
  \"fontes\": [
    {
      \"tipo\": \"[livro | artigo | site | vídeo | filme]\",
      \"referencia\": \"[título, autor, link ou identificação da fonte]\"
    }
  ]
}

Não invente fontes inexistentes. Caso não tenha fontes específicas, indique caminhos gerais de pesquisa (ex: bases de dados, autores reconhecidos, instituições)."""
)

In [17]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class SeletiveOut(BaseModel):
  pergunta: str = Field(description="Pergunta do usuário")
  resposta: str = Field(description="Resposta da IA")
  fontes: List[Dict[Literal["livro", "artigo", "site", "vídeo", "filme"], str]]

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1,
    api_key=API_KEY,
)

In [18]:
from langchain_core.messages import HumanMessage, SystemMessage

Seletive_chain = llm.with_structured_output(SeletiveOut)

def seletive(mensagem: str) -> Dict:
  saida: SELETIVE_PROMPT = Seletive_chain.invoke([
      SystemMessage(content=SELETIVE_PROMPT),
      HumanMessage(content=mensagem)
  ])

  return saida.model_dump()

In [21]:
test = ["O que é sociologia ?"]
for i in test:
  print(seletive(i))

{'pergunta': 'O que é sociologia ?', 'resposta': 'A sociologia é a ciência que se dedica ao estudo da sociedade humana, suas estruturas, relações sociais, interações e processos culturais. Ela busca compreender como os indivíduos e os grupos sociais se organizam, agem, pensam e se transformam ao longo do tempo. \n\n**Principais Objetivos:**\n*   **Analisar padrões sociais:** Identificar regularidades e tendências no comportamento humano e nas relações sociais.\n*   **Investigar instituições sociais:** Estudar como instituições como a família, o Estado, a escola, a religião e a economia moldam a vida social.\n*   **Compreender a desigualdade:** Examinar as causas e consequências das desigualdades sociais (classe, gênero, raça, etc.).\n*   **Explicar a mudança social:** Analisar os fatores que levam às transformações na sociedade, como movimentos sociais, avanços tecnológicos e globalização.\n\n**Métodos de Estudo:**\nOs sociólogos utilizam uma variedade de métodos de pesquisa, tanto qua